# On the fly Cholesky 

In [1]:
"""Build Benzene molecule with a minimal GTO basis, using PYSCF
"""
%load_ext autoreload
%autoreload 2

# Generic imports
import numpy as np
from pathlib import Path


In [2]:
""" Compute wave functions, grid and density using PYSCF
"""
from isdf_prototypes.clean_isdf import benzene_from_pyscf

output_root = Path('../outputs/cholesky')
n_grid_points = [10, 10, 10]
data: dict = benzene_from_pyscf(output_root, n_grid_points)
print("Returned data:", data.keys())
    
# Add real-space points, and volume element
data.update({'grid_points': data['cube_grid'].get_coords()})
data.update({'dV': data['cube_grid'].get_volume_element()})


converged SCF energy = -229.930646528207
Returned data: dict_keys(['wfs', 'rho', 'cube_grid', 'n_occ', 'dV'])


## Cholesky Algorithm Outline

This outlines the schematic pivoted Cholesky algorithm. Because the corresponding Gram matrix has shape $(np, np)$, we absolutely want to avoid storing this array in Octopus, so we use an on-the-fly version of the algorithm.

This will scale like $\mathcal{O}(n_p k N_e)$, where $n_p$ is the grid size, $k$ is the number of pivot points (which become our interpolation points), and $N_e$ is the number of occupied KS states (for molecular systems).

The generic expressions for the definition of $L$ can be found on [wikipedia](https://en.wikipedia.org/wiki/Cholesky_decomposition#The_Cholesky–Banachiewicz_and_Cholesky–Crout_algorithms)

This [paper](https://pubs.acs.org/doi/10.1021/acs.jctc.3c00615) gives a reference of CD w.r.t. ISDF, but the notation differs from the ISDF sources I follow.


### The Gram Matrix of the Pair Products

The Gram matrix for real systems is defined as:

$$
G_{r,p} = \sum_{ij} \phi_i(\mathbf{r}) \phi_j(\mathbf{r}) \phi_i(\mathbf{p}) \phi_j(\mathbf{r})
$$

where $r$ and $p$ are indices over grid points. In the on-the-fly algorithm, $p$ is defined as the pivot point, and so, one only computes the element $G_{r,p}$ per grid point of the inner loop (see below)

Because the pair products have a separable form, this can be rewritten as:

$$
G_{r,p} = \left[\sum_{i} \phi_i(\mathbf{r}) \phi_i(\mathbf{p}) \right] 
   \left[\sum_{j}  \phi_j(\mathbf{r}) \phi_j(\mathbf{p}) \right]
$$

reducing the scaling from $\mathcal{O}(N^2_e)$ to $\mathcal{O}(N_e)$ per grid point $r$.

The diagonal of the Gram matrix simplifies to:

$$
G_{r,r} = G_{r} = \left[\sum_{i} \phi_i(\mathbf{r})^2 \right] 
   \left[\sum_{j}  \phi_j(\mathbf{r})^2 \right]
$$


### Algorithm Outline

Note, fortran is column-major and python (numpy) is row-major. The fastest index will swap when moving from python to fortran.

```text
subroutine pivtoed_cholsky_on_the_fly(st, k_max, thres, pivots):
    
    k_max <- Maximum number of pivot points
    Allocate L(np, k_max), diag_G(np), pivots(k_max), residuals(np)

    ! Initialise diagonal of the Gram matrix
    do ip = 1, np
       ! Implement according to the expression above
       diag_G(ip) = gram_matrix_diagonal(st, n_occ)
    enddo

    do k = 1, k_max
        ! Choose the pivot index
        i_pivot = argmax(diag_G)
        pivots(k) = diag_G(i_pivot)
        if (diag_G(i_pivot) < thres) return pivots(1:k)

        ! Set the pivot value of the L matrix
        L(i_pivot, k) = sqrt(diag_G(i_pivot))
        inv_L = 1.0 / L(i_pivot, k) 

        ! Define the residuals
        if k == 1:
          residuals = 0.0
        else
          DGEMV L(1:np, 1:k-1), transpose(L(i_pivot, 1:k-1))
          -> residual(1:np)
        endif

        ! Set all other rows of column k
        do ip = 1, np
          ! Avoid overwriting the above, L(i_pivot, k)
          if ip == i_pivot: cycle
            
          ! Compute G on-the-fly, as defined in the expression above
          G_rp = gram_matrix_element(ip, i_pivot, st)

          L(ip, k) = inv_L * (G_rp - residual(ip)) 
          
          ! Update the diagonal
          diag_G(ip) = diag_G(ip) - L(ip, k)**2
        enddo
   enddo

end subroutine     
```

Once done, we can use the existing code in this repository to construct the ISDF vectors, and quantify the error w.r.t. the exact wave function product.


In [3]:
""" Cholesky implementation
"""
# Total number of grid points
n_total = data['grid_points'].shape[0]
# Total number of states
n_states = data['wfs'].shape[1]

assert data['n_occ'] == 21, "Expected occupied states for benzene with this basis is 21"
assert data['wfs'].shape == (n_total, n_states), "Wavefunctions (MOs) array has shape (np, n_states)"


# TODO Implement the above algorithm
# - Modify API as necessary
def choleksy_decomposition_with_pivoting(wfs, n_interp):
    raise NotImplementedError("Implement choleksy_decomposition_with_pivoting")
    # Return the pivot indices
    return []


In [4]:
""" L.H.S. Full product basis matrix. The functions we want to interpolate
"""
from isdf_prototypes.math_ops import face_splitting_product

# Construct product basis matrix
z = face_splitting_product(data['wfs'])
assert z.shape == (n_total, data['n_occ']**2)

In [5]:
""" R.H.S.: Interpolation points and vectors, and approximate product matrix
"""
from typing import Tuple

from isdf_prototypes.isdf_vectors import construct_interpolation_vectors_matrix


def compute_approximate_product_matrix(wfs, n_interp) -> Tuple[np.ndarray, int]:
    """ Compute an approximate product matrix using ISDF interpolation vectors
     
      z_{\text{isdf}} = \sum_\mu^{N_\mu} \zeta_{\mu}(\mathbf{r}) \phi_i(\mathbf{r}_\mu) \phi_j(\mathbf{r}_\mu)
    
    which can be represented as the matrix equation:
        
      Z_{\text{isdf}} = \Zeta (Phi \bullet \Psi), 
    
    where (Phi \bullet \Psi) is the face-splitting product.
    
    :param wfs: Wave functions on real-space grid, of shape(n_points, nocc**2)
    :param n_interp:  Number of ISDF vectors
    :return: 
    """
    # Avoid using global data
    total_grid_size = wfs.shape[0]
    n_products = wfs.shape[1]**2

    # Construct the interpolation points
    indices = choleksy_decomposition_with_pivoting(data['wfs'], n_interp)
    assert len(indices) == n_interp
    
    # Construct the interpolation points vectors
    zeta = construct_interpolation_vectors_matrix(wfs, indices)
    assert zeta.shape == (total_grid_size, n_interp)
    
    # Product basis defined on the interpolation grid
    z_interp = face_splitting_product(wfs[indices, :])
    assert z_interp.shape == (n_interp, n_products)
    
    # ISDF approximation to the product basis
    # i.e. perform \sum_\mu^{N_\mu} \zeta_{\mu}(\mathbf{r}) \phi_i(\mathbf{r}_\mu) \phi_j(\mathbf{r}_\mu)
    # using matrix-matrix product
    z_isdf = zeta @ z_interp
    assert z_isdf.shape == (total_grid_size, n_products)
    
    return z_isdf
    

def error_l2(f1, f2, dV) -> dict:
    """  Error using L2 metric.
    
    :param f1: 
    :param f2: 
    :param grid: 
    :return: 
    """
    assert dV > 0.0, "Must have a finite volume element"
    diff = f1 - f2
    # Integrate over grid points
    err_ij = np.sum(diff * diff, axis=0) * dV
    # Paper says ^2, but I think this should be RMSE, and therefore be ^1/2
    # NOTE, THIS COMPLETELY CHANGES THE ERROR, and really makes it such that one needs ~ 100 points
    # such that the max error is reasonable. However, for 100 points the plot comparisons 
    # below look entirely consistent
    err_ij = np.sqrt(err_ij)
    err = {'min': np.amin(err_ij), 'max': np.amax(err_ij), 'mean': np.mean(err_ij)}

    return err
    

def relative_error_l2(f1, f2, dV):
    """
    :param f1:  Target function
    :param f2:  Approximate function
    :param grid: system grid
    """
    mean_err = error_l2(f1, f2, dV)['mean']
    # Volume element required in discretisation of <f1, f1>
    norm_f1 = np.sqrt(dV * np.sum(f1**2, axis=0))
    mean_norm_f1 = np.mean(norm_f1)
    return mean_err / mean_norm_f1
    

In [6]:
""" Run the error in the expansion of pair products using the ISDF vectors

From my prior notebook (isdf_vectors.py), these were the typical errors one got on benzene (same functional) using QR decomposition on a randomly sampled matrix:
9   {'min': 9.345572027838745e-05,  'max': 0.00251585330283558,   'mean': 0.0010730809703227925}  rel. 2-error 0.7328820171707147
25  {'min': 3.8039235072977515e-05, 'max': 0.002069107549818602,  'mean': 0.0006128654717785342}  rel. 2-error 0.41856867807116455
49  {'min': 1.4698690853164507e-05, 'max': 0.0010400652001405033, 'mean': 0.00025373877194010184} rel. 2-error 0.1732959470504291
81  {'min': 3.4549130138913135e-06, 'max': 0.00025182121449866877,'mean': 2.560415693257912e-05}  rel. 2-error 0.0174868688381078
100 {'min': 1.300363082060416e-06,  'max': 3.360789700461351e-05, 'mean': 6.212559011270438e-06}  rel. 2-error 0.004242990888751269

"""
# Number of interpolation vectors. Set as desired
n_interpolation_vectors = [10, 25, 50, 100]

for n in n_interpolation_vectors:
    z_isdf, n_interp = compute_approximate_product_matrix(data['wfs'], n)
    print(n_interp, error_l2(z, z_isdf, data['dV']), "rel. 2-error", relative_error_l2(z, z_isdf, data['dV']))

Implement choleksy_decomposition_with_pivoting


AssertionError: 